In [115]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
import random
from statsmodels.stats.proportion import binom_test
from statsmodels.stats.multitest import multipletests
#from sys import argv
from argparse import ArgumentParser 

In [160]:
def btest(pa1, pa2, seed=0, return_proportions=False):
    """ Performs genome wide binomial test between two groups of taxa
    Parameters
    ----------
    df1 : pd.DataFrame
        Rows are taxa, columns are genes
    df2 : pd.DataFrame
        Rows are taxa, columns are genes
    Returns
    -------
    pd.Series : list of genes associated with df1
    pd.Series : list of genes associated with df2
    """
    np.random.seed(seed)
    random.seed(seed)
    #pa1 = df1 > 0
    #pa2 = df2 > 0
    idx = list(set(pa1.columns) | set(pa2.columns))
    idx.sort()
    pa1 = pa1.sum(axis=0).reindex(idx).fillna(0)
    pa2 = pa2.sum(axis=0).reindex(idx).fillna(0)
    n = pa1 + pa2
    #print("min n", min(list(n.values)))
    obs = list(zip(list(pa1.values), list((pa2.values + 1) / (pa2 + 1).sum()), list(n.values)))
    pvals = pd.Series([binom_test(_a, _n, _b, 'two-sided') for (_a, _b, _n) in obs],
                      index=n.index)
    if return_proportions:
        res = pd.DataFrame({'groupA': pa1, 'groupB': pa2, 'pval': pvals})
        def relabel_f(x):
            if x['groupA'] < x['groupB']:
                return 'groupB'
            else:
                return 'groupA'
        res['side'] = res.apply(relabel_f, axis=1)
        return res

    return pvals


def parse_genome(df):
    genome_id = df['#query'][0].split('_')[0]
    keggs = df['KEGG_ko'].replace('-', None).dropna()
    keggs = list(map(lambda x: x.split(','), keggs.values))
    keggs = sum(keggs, [])
    keggs = pd.DataFrame({'KEGG_ko': keggs})
    keggs['genome_id'] = genome_id
    return keggs


def to_sparse_matrix(func_df, genome_id='genome_id', kegg_id='KEGG_ko'):
    # create genome-specific index
    ogus = list(set(func_df[genome_id]))
    ogu_lookup = pd.Series(np.arange(0, len(ogus)), ogus)
    # create KEGG-specific index
    keggs = list(set(func_df[kegg_id]))
    kegg_lookup = pd.Series(np.arange(0, len(keggs)), keggs)
    # rename names as numbers
    ogu_id = func_df[genome_id].apply(lambda x: ogu_lookup.loc[x]).astype(np.int64)
    kegg_id = func_df[kegg_id].apply(lambda x: kegg_lookup.loc[x]).astype(np.int64)
    # assign the presence / absence of a gene
    func_df['count'] = 1
    c = func_df['count'].values
    # format into a matrix
    data = coo_matrix((c, (ogu_id, kegg_id)))
    ko_ogu = pd.DataFrame(data.todense(), index=ogus, columns=keggs)
    return ko_ogu
 
    
def _test(X, y, z, k, G, nc, pt):
    """
    X: count matrix of microbes (m microbes X n samples)
    y: disease lables for samples (n samples)
    z: colname of the condition (eg. disease status)
    k: top k microbes, eg k = 100
    G: annotation matrix of microbes (microbe ids X kegg ids), and have a column with species names
    nc: number of cpu
    pt: the threshold of p value, eg pt = 0.001
    """
    # Convert counts and group labels to PyDESeq2 input format
    dds = DeseqDataSet(
        X, 
        y, 
        design_factors = z,# compare samples based on the "disease"
        refit_cooks = True,
        n_cpus = nc
    )
    dds.deseq2()
    #set of the sample status: Healthy, disease
    status = set(y[z])
    #remove healthy
    status.remove("Healthy")
    disease = ', '.join(status)
    #sample status colname: disease, 
    #disease: name of that disease
    stat_res = DeseqStats(dds,contrast = [z, disease, "Healthy" ], n_cpus=nc)
    #stat_res = DeseqStats(dds,contrast = ["disease", "Healthy", "AD" ], n_cpus=8)
    res = stat_res.summary()
    res_df = stat_res.results_df    
    res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
    res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
    #top k microbes: the ones with largest CI_5
    top = res_df.sort_values(by=['CI_5'],ascending=False).head(k)
    #bottom k microbes: the ones with smallest CI_95
    bot = res_df.sort_values(by=['CI_95'],ascending=True).head(k)
    #convert microbe names to species ids
    top_microbe = set(top.index)
    bot_microbe = set(bot.index)
    top_gene_matrix = G[G['Species'].isin(top_microbe)]
    bot_gene_matrix = G[G['Species'].isin(bot_microbe)]
    frames = [top_gene_matrix, bot_gene_matrix]
    #concatenate two gene matrix and filter out the kegg with only 0 values
    Conta_gene_matrix = pd.concat(frames)
    Conta_gene_matrix = Conta_gene_matrix.loc[:, (Conta_gene_matrix != 0).any(axis=0)]
    top_gene_matrix = Conta_gene_matrix[Conta_gene_matrix['Species'].isin(top_microbe)]
    bot_gene_matrix = Conta_gene_matrix[Conta_gene_matrix['Species'].isin(bot_microbe)]
    top_gene_matrix = top_gene_matrix.drop('Species', axis=1)
    bot_gene_matrix = bot_gene_matrix.drop('Species', axis=1)
    #binomial test Return number of genes elevated in top
    #C = btest(G[top], G[bot])
    kegg = btest(top_gene_matrix,bot_gene_matrix,return_proportions=True)
    kegg_top = kegg.loc[kegg['side'] == 'groupA']
    kegg_top_sig = kegg_top.loc[kegg_top['pval'] <= pt]
    C = len(kegg_top_sig.index)
    return C, kegg_top


def permutation_test(X, y, z, k, p, G, nc, pt):
    #p: number of permutations, eg p = 1000
    T = _test(X, y, z, k, G, nc, pt)
    T_list = np.zeros(p)
    for i in range(p):
        #shuffle the group lables
        y_permutated = np.random.permutation(y[z])
        y_permutated = pd.DataFrame(y_permutated, index=y.index)
        y_permutated.columns = [z]
        y_permutated.reindex(X.index)
        T_ = _test(X, y_permutated, z, k, G, nc, pt)
        T_list[i] = T_
    p_value = np.sum(T_list > T) / (p+1)
    return T, p_value


def get_options():
    parser = ArgumentParser()
    parser.add_argument("-x", dest="microbe_table",
                       help="count matrix of microbes (m microbes X n samples)",
                       required=True)
    parser.add_argument("-y", dest="disease_labels",
                       help="disease lables for samples (n samples)",
                       required=True)
    parser.add_argument("-z", dest="disease_status",
                       help="colnames of the diseases status",
                       required=True)
    parser.add_argument("-k", dest="top_k", default=100, type=int,
                       help="top k microbes. "
                            "Default: %(default)i")
    parser.add_argument("-p", dest="permutations", default=1000, type=int,
                       help="number of permutations. "
                            "Default: %(default)i")
    parser.add_argument("-G", dest="gene_matrix",
                       help="gene matrix for microbes",
                       required=True)
    parser.add_argument("-nc", dest="number_cpus",default=8, type=int,
                       help="number of cpus")
    parser.add_argument("-pt", dest="pval",default=0.05, type=float,
                       help="threshold of p value")
    parser.add_argument("-o", dest="output_file",
                       help="number of genes and p value",
                       required=True)
    options = parser.parse_args()
    return options


def main():
    option = get_options()
    input_table = pd.read_table(option.microbe_table, sep = '\t', index_col = 0)
    input_table2 = pd.read_table(option.disease_labels, sep = '\t', index_col = 'featureid')
    input_table3 = pd.read_table(option.gene_matrix, sep='\t', index_col = 0 )

    a,b = permutation_test(X = input_table,
                           y = input_table2,
                           z = option.disease_status,
                           k = option.top_k,
                           p = option.permutations,
                           G = input_table3,
                           nc = option.number_cpus,
                           pt = option.pval)

    with open(option.output_file, "w") as output_h:
        output_h.write(str(a) + "\n")
        output_h.write(str(b) + "\n")


In [155]:
input_table = pd.read_table('../../permutation_table/table_sim_microbes.txt', sep = '\t', index_col = 0)
input_table2 = pd.read_table('../../permutation_table/metadata_sim.txt', sep = '\t', index_col = 'featureid')
input_table4 = pd.read_table('../../permutation_table/G_matrix_sim.txt', sep='\t', index_col = 0 )

In [165]:
input_table2

,Unnamed: 0,disease,Study,gender,age,Batch
featureid,,,,,,
S1,0,AD,Laske2022AD,Female,58.0,1
S2,1,AD,Laske2022AD,Female,78.0,1
S3,2,AD,Laske2022AD,Female,73.0,1
S4,3,AD,Laske2022AD,Male,77.0,1
S5,4,AD,Laske2022AD,Female,73.0,1
S6,5,Healthy,Laske2022AD,Male,79.0,1
S7,6,Healthy,Laske2022AD,Male,76.0,1
S8,7,Healthy,Laske2022AD,Female,78.0,1
S9,8,Healthy,Laske2022AD,Female,75.0,1


In [156]:
input_table

,M1,M2,M3,M4
S1,10,12,1,1
S2,10,13,2,2
S3,10,13,1,1
S4,10,14,1,1
S5,10,15,1,1
S6,1,1,20,13
S7,1,2,15,11
S8,1,1,11,14
S9,1,0,12,16
S10,1,1,11,18


In [149]:
#pd.read_table('../../permutation_table/table_AD_microbes.txt', sep = '\t', index_col = 0)

In [164]:
input_table4

,Species,ko:K00001,ko:K00002,ko:K00003,ko:K00004
MGYG01,M1,2,0,3,0
MGYG02,M2,3,0,1,0
MGYG03,M3,0,2,0,1
MGYG04,M4,0,5,0,2


In [137]:
np.random.permutation(y[z])

array(['sick', 'Healthy', 'sick', 'Healthy', 'sick', 'Healthy', 'Healthy',
       'sick', 'sick', 'Healthy'], dtype=object)

In [161]:
a,b = _test(X = input_table, y = input_table2, z = 'disease', k = 2, G = input_table4, nc = 8, pt = 0.05)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.02 seconds.

Fitting LFCs...
... done in 0.02 seconds.

Refitting 0 outlier genes.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


/Users/dongminjin/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/dongminjin/PyDESeq2/pydeseq2/utils.py:557: RuntimeWarning: invalid value encountered in log
  x0=[np.log(alpha_hat)],
/Users/dongminjin/PyDESeq2/pydeseq2/utils.py:534: RuntimeWarning: invalid value encountered in log
  reg += (np.log(alpha) - np.log(alpha_hat)) ** 2 / (2 * prior_disp_var)
/Users/dongminjin/PyDESeq2/pydeseq2/utils.py:552: RuntimeWarning: invalid value encountered in log
  reg_grad += (np.log(alpha) - np.log(alpha_hat)) / (alpha * prior_disp_var)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
M1,17.841389,6.617917,0.699453,9.461561,3.033784e-21,6.067568e-21
M2,23.931540,7.052666,0.694752,10.151343,3.268344e-24,1.307338e-23
M3,4.313172,-0.267145,0.614052,-0.435052,6.635245e-01,6.635245e-01
M4,4.436731,-0.328545,0.613027,-0.535939,5.920009e-01,6.635245e-01


In [162]:
a

2

In [163]:
b

,groupA,groupB,pval,side
ko:K00001,5,0,0.000002,groupA
ko:K00003,4,0,0.000026,groupA


In [158]:
a, b = permutation_test(X = input_table, y = input_table2, z = 'disease', k = 2, p = 10, G = input_table4, nc = 8, pt = 0.05)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.00 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 0 outlier genes.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


/Users/dongminjin/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/dongminjin/PyDESeq2/pydeseq2/utils.py:557: RuntimeWarning: invalid value encountered in log
  x0=[np.log(alpha_hat)],
/Users/dongminjin/PyDESeq2/pydeseq2/utils.py:534: RuntimeWarning: invalid value encountered in log
  reg += (np.log(alpha) - np.log(alpha_hat)) ** 2 / (2 * prior_disp_var)
/Users/dongminjin/PyDESeq2/pydeseq2/utils.py:552: RuntimeWarning: invalid value encountered in log
  reg_grad += (np.log(alpha) - np.log(alpha_hat)) / (alpha * prior_disp_var)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
M1,17.841389,6.617917,0.699453,9.461561,3.033784e-21,6.067568e-21
M2,23.931540,7.052666,0.694752,10.151343,3.268344e-24,1.307338e-23
M3,4.313172,-0.267145,0.614052,-0.435052,6.635245e-01,6.635245e-01
M4,4.436731,-0.328545,0.613027,-0.535939,5.920009e-01,6.635245e-01


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.00 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 0 outlier genes.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
M1,17.841389,1.028832,1.599435,0.643247,0.520064,0.762359
M2,23.931540,0.971416,1.717960,0.565448,0.571769,0.762359
M3,4.313172,-0.392198,0.350042,-1.120430,0.262530,0.762359
M4,4.436731,0.051989,0.480322,0.108239,0.913806,0.913806


...

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
M1,17.841389,1.028832,1.599435,0.643247,0.520064,0.762359
M2,23.931540,0.971416,1.717960,0.565448,0.571769,0.762359
M3,4.313172,-0.392198,0.350042,-1.120430,0.262530,0.762359
M4,4.436731,0.051989,0.480322,0.108239,0.913806,0.913806


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.00 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 0 outlier genes.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
M1,17.841389,1.028832,1.599435,0.643247,0.520064,0.762359
M2,23.931540,0.971416,1.717960,0.565448,0.571769,0.762359
M3,4.313172,-0.392198,0.350042,-1.120430,0.262530,0.762359
M4,4.436731,0.051989,0.480322,0.108239,0.913806,0.913806


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.00 seconds.

Fitting dispersion trend curve...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.00 seconds.

Fitting LFCs...
... done in 0.00 seconds.

Refitting 0 outlier genes.

Running Wald tests...
... done in 0.00 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
M1,17.841389,1.028832,1.599435,0.643247,0.520064,0.762359
M2,23.931540,0.971416,1.717960,0.565448,0.571769,0.762359
M3,4.313172,-0.392198,0.350042,-1.120430,0.262530,0.762359
M4,4.436731,0.051989,0.480322,0.108239,0.913806,0.913806


In [159]:
a, b

(2, 0.0)

In [60]:
metadata = pd.read_table('../../permutation_table/eggNOG_species_rep.txt', sep = '\t', index_col = 'Species_rep')

In [62]:
new_gene_matrix = pd.merge(metadata, input_table4, left_index=True, right_index=True)

In [87]:
G_new = new_gene_matrix.drop_duplicates()

In [88]:
G_new

,Species,ko:K09121,ko:K14063,ko:K06122,ko:K14155,ko:K02670,ko:K14089,ko:K11908,ko:K01641,ko:K12700,...,ko:K00670,ko:K02637,ko:K02358,ko:K18700,ko:K02922,ko:K05798,ko:K01126,ko:K15654,ko:K01190,ko:K06948
MGYG000000001,GCA-900066495 sp902362365,2,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,2,0,2,0
MGYG000000002,Blautia_A faecis,1,0,0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,2,0
MGYG000000003,Alistipes shahii,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,7,0,7,0
MGYG000000004,Anaerotruncus colihominis,1,0,0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
MGYG000000005,Terrisporobacter glycolicus_A,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MGYG000004901,NaN,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MGYG000004902,An23 sp900545755,3,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
MGYG000004903,Blautia sp900555025,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,0,4,0
MGYG000004904,NaN,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [75]:
G_new.set_index('Species', inplace=True)

In [54]:
input_table3[input_table3['Species_rep'].isin(dic)].drop_duplicates()

,Species,Species_rep
0,GCA-900066495 sp902362365,MGYG000000001
1,Blautia_A faecis,MGYG000000002
2,Alistipes shahii,MGYG000000003
3,Anaerotruncus colihominis,MGYG000000004
4,Terrisporobacter glycolicus_A,MGYG000000005
...,...,...
4861,NaN,MGYG000004891
4865,NaN,MGYG000004895
4868,Weissella confusa_B,MGYG000004898
4871,NaN,MGYG000004901


In [51]:
gene_matrix = input_table3.filter(items = dic, axis=0)
gene_matrix

,Species,Species_rep


In [36]:
', '.join(status)

'AD'

In [18]:
    X = input_table
    y = input_table2
    z = 'disease'
    M = input_table3
    G = input_table4
    k = 2
    dds = DeseqDataSet(
        X, 
        y, 
        design_factors = z,# compare samples based on the "disease"
        refit_cooks = True,
        n_cpus = 8
    )
    dds.deseq2()
    #set of the sample status: Healthy, disease
    status = set(y[z])
    #remove healthy
    status.remove("Healthy")
    disease = ', '.join(status)
    #sample status colname: disease, 
    #disease: name of that disease
    stat_res = DeseqStats(dds,contrast = [z, disease, "Healthy" ], n_cpus=8)
    #stat_res = DeseqStats(dds,contrast = ["disease", "Healthy", "AD" ], n_cpus=8)
    res = stat_res.summary()
    res_df = stat_res.results_df    
    res_df["CI_95"] = res_df["log2FoldChange"] + res_df['lfcSE'] * 1.96
    res_df["CI_5" ] = res_df["log2FoldChange"] - res_df['lfcSE'] * 1.96
    #top k microbes: the ones with largest CI_5
    top = res_df.sort_values(by=['CI_5'],ascending=False).head(k)
    #bottom k microbes: the ones with smallest CI_95
    bot = res_df.sort_values(by=['CI_95'],ascending=True).head(k)
    #convert microbe names to species ids
    top_microbe = set(top.index)
    bot_microbe = set(bot.index)
    top_rep = M[M['Species'].isin(top_microbe)]
    bot_rep = M[M['Species'].isin(bot_microbe)]
    top_id = top_rep['Species_rep'].drop_duplicates()
    top_id = [_test_top_id for _test_top_id in top_id if "." not in _test_top_id]
    bot_id = bot_rep['Species_rep'].drop_duplicates()
    bot_id = [_test_bot_id for _test_bot_id in bot_id if "." not in _test_bot_id]


Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.40 seconds.

Fitting dispersion trend curve...
... done in 0.41 seconds.

Fitting MAP dispersions...
... done in 0.43 seconds.

Fitting LFCs...
... done in 0.34 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.22 seconds.

Fitting MAP dispersions...
... done in 0.22 seconds.

Fitting LFCs...
... done in 0.18 seconds.

Running Wald tests...
... done in 0.32 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
1XD42-69 sp014287635,2130.637601,0.570896,0.257603,2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,0.668411,1.372424,0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,0.486055,0.244547,1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,-3.479790,0.637017,-5.462633,4.691239e-08,0.000006
51-20 sp900539605,97.994360,-1.575257,0.887438,-1.775061,7.588780e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,1.521263,0.608664,2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,0.730225,0.438464,1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,1.101342,0.803632,1.370455,1.705449e-01,0.465896
Zag111 sp900551965,88.867373,-0.648109,0.514632,-1.259364,2.078987e-01,0.516096


In [90]:
top_microbe

{'Acidaminococcus intestini', 'Succinivibrio sp000431835'}

In [81]:
top_gene_matrix = G.filter(items = top_id, axis=0)
top_gene_matrix

,ko:K09121,ko:K14063,ko:K06122,ko:K14155,ko:K02670,ko:K14089,ko:K11908,ko:K01641,ko:K12700,ko:K03572,...,ko:K00670,ko:K02637,ko:K02358,ko:K18700,ko:K02922,ko:K05798,ko:K01126,ko:K15654,ko:K01190,ko:K06948
MGYG000000412,0,0,0,2,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
MGYG000001440,1,0,0,2,0,0,0,0,0,1,...,0,0,2,0,0,0,0,0,0,0


In [92]:
G_new.to_csv('../../permutation_table/G_matrix_new.txt', sep = '\t')

In [95]:
pd.read_table('../../permutation_table/G_matrix_new.txt', sep='\t', index_col = 0 )

,Species,ko:K09121,ko:K14063,ko:K06122,ko:K14155,ko:K02670,ko:K14089,ko:K11908,ko:K01641,ko:K12700,...,ko:K00670,ko:K02637,ko:K02358,ko:K18700,ko:K02922,ko:K05798,ko:K01126,ko:K15654,ko:K01190,ko:K06948
MGYG000000001,GCA-900066495 sp902362365,2,0,1,1,0,0,0,0,1,...,0,0,1,0,0,0,2,0,2,0
MGYG000000002,Blautia_A faecis,1,0,0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,2,0
MGYG000000003,Alistipes shahii,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,7,0,7,0
MGYG000000004,Anaerotruncus colihominis,1,0,0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
MGYG000000005,Terrisporobacter glycolicus_A,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MGYG000004901,NaN,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MGYG000004902,An23 sp900545755,3,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
MGYG000004903,Blautia sp900555025,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,0,4,0
MGYG000004904,NaN,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [99]:
# top_gene_matrix = G_new.filter(items = top_microbe, axis=0)
# top_gene_matrix

top_G = G_new[G_new['Species'].isin(top_microbe)]
#top_G = top_G.drop('Species', axis=1)
top_G

,Species,ko:K09121,ko:K14063,ko:K06122,ko:K14155,ko:K02670,ko:K14089,ko:K11908,ko:K01641,ko:K12700,...,ko:K00670,ko:K02637,ko:K02358,ko:K18700,ko:K02922,ko:K05798,ko:K01126,ko:K15654,ko:K01190,ko:K06948
MGYG000000412,Succinivibrio sp000431835,0,0,0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
MGYG000001440,Acidaminococcus intestini,1,0,0,2,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0


In [102]:
T_list = np.zeros(10)
T_list

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [104]:
T_list[1] = 5
T_list

array([0., 5., 0., 0., 0., 0., 0., 0., 0., 0.])

In [106]:
T = 4
np.sum(T_list > T)

1

In [107]:
T_list > T

array([False,  True, False, False, False, False, False, False, False,
       False])

In [30]:
    top_gene_matrix = G.filter(items = top_id, axis=0)
    bot_gene_matrix = G.filter(items = bot_id, axis=0)
    frames = [top_gene_matrix, bot_gene_matrix]
    Conta_gene_matrix = pd.concat(frames)
    Conta_gene_matrix = Conta_gene_matrix.loc[:, (Conta_gene_matrix != 0).any(axis=0)]
    top_gene_matrix = Conta_gene_matrix.filter(items = top_id, axis=0)
    bot_gene_matrix = Conta_gene_matrix.filter(items = bot_id, axis=0)


In [26]:
Conta_gene_matrix

,ko:K09121,ko:K14155,ko:K03572,ko:K07460,ko:K00287,ko:K03110,ko:K19955,ko:K01872,ko:K07016,ko:K01681,...,ko:K07391,ko:K00528,ko:K11016,ko:K03431,ko:K01715,ko:K11102,ko:K03527,ko:K02358,ko:K01126,ko:K01190
MGYG000000412,0,2,1,0,1,1,0,1,1,0,...,0,0,0,1,0,0,1,1,0,0
MGYG000001440,1,2,1,0,0,1,4,2,0,3,...,1,0,0,1,1,1,1,2,0,0
MGYG000000700,0,2,1,1,0,1,0,1,0,0,...,1,0,4,0,0,0,1,0,6,5
MGYG000000941,0,2,1,1,1,1,1,1,0,0,...,1,1,0,0,0,0,1,1,1,1


In [33]:
top_gene_matrix

,ko:K09121,ko:K14155,ko:K03572,ko:K07460,ko:K00287,ko:K03110,ko:K19955,ko:K01872,ko:K07016,ko:K01681,...,ko:K07391,ko:K00528,ko:K11016,ko:K03431,ko:K01715,ko:K11102,ko:K03527,ko:K02358,ko:K01126,ko:K01190
MGYG000000412,0,2,1,0,1,1,0,1,1,0,...,0,0,0,1,0,0,1,1,0,0
MGYG000001440,1,2,1,0,0,1,4,2,0,3,...,1,0,0,1,1,1,1,2,0,0


In [31]:
bot_gene_matrix

,ko:K09121,ko:K14155,ko:K03572,ko:K07460,ko:K00287,ko:K03110,ko:K19955,ko:K01872,ko:K07016,ko:K01681,...,ko:K07391,ko:K00528,ko:K11016,ko:K03431,ko:K01715,ko:K11102,ko:K03527,ko:K02358,ko:K01126,ko:K01190
MGYG000000700,0,2,1,1,0,1,0,1,0,0,...,1,0,4,0,0,0,1,0,6,5
MGYG000000941,0,2,1,1,1,1,1,1,0,0,...,1,1,0,0,0,0,1,1,1,1


In [13]:
a,b = _test(X=input_table, y=input_table2, z='disease', k=2, M=input_table3, G=input_table4)

Fitting size factors...
... done in 0.01 seconds.

Fitting dispersions...
... done in 0.41 seconds.

Fitting dispersion trend curve...
... done in 0.40 seconds.

Fitting MAP dispersions...
... done in 0.41 seconds.

Fitting LFCs...
... done in 0.34 seconds.

Refitting 1806 outlier genes.

Fitting dispersions...
... done in 0.21 seconds.

Fitting MAP dispersions...
... done in 0.21 seconds.

Fitting LFCs...
... done in 0.17 seconds.

Running Wald tests...
... done in 0.31 seconds.

Log2 fold change & Wald test p-value: disease AD vs Healthy


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
1XD42-69 sp014287635,2130.637601,0.570896,0.257603,2.216188,2.667864e-02,0.158735
28L sp000177555,2.246988,0.668411,1.372424,0.487030,6.262371e-01,NaN
43-108 sp001915545,500.585113,0.486055,0.244547,1.987571,4.685920e-02,0.231489
51-20 sp001917175,9710.835303,-3.479790,0.637017,-5.462633,4.691239e-08,0.000006
51-20 sp900539605,97.994360,-1.575257,0.887438,-1.775061,7.588780e-02,0.299433
...,...,...,...,...,...,...
Zag111 sp002103105,41.825340,1.521263,0.608664,2.499349,1.244218e-02,0.098247
Zag111 sp003258735,51.116899,0.730225,0.438464,1.665416,9.582989e-02,0.340741
Zag111 sp004558955,17.609086,1.101342,0.803632,1.370455,1.705449e-01,0.465896
Zag111 sp900551965,88.867373,-0.648109,0.514632,-1.259364,2.078987e-01,0.516096


min n 0.0


ValueError: n must be an integer not less than 1